# Light_GCN

## Import

In [1]:
import torch
from torch import nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import scipy.sparse as sp
import time 
import pickle

## Config

In [3]:
cfg = {
    'batch_size' : 4096,
    'test_batch_size' : 32,
    'dataset_dir_path' : "./dataset/lgcn",
    'dataset_file_name' : "user_problem_mat.csv",
    'adj_mat_name' : "s_pre_adj_mat.npz",
    'd_mat_name' : "d_mat.npz",
    'embedding_dim' : 10,
    'n_layers' : 3,
    'pretrain' : False,
    'dropout' : False,
    'A_split' : False,
    'device' : 'cuda',
    'decay' : 1e-4,
    'lr' : 1e-3,
    'model_file_path' : './saved_model/lgcn_weight_10.pt',
    'topks' : [10,20],
}

if cfg['pretrain']:
    try:
        model = torch.load(cfg['model_file_path'])
        cfg['user_emb'] = model['embedding_user.weight']
        cfg['item_emb'] = model['embedding_item.weight']
    except:
        print('there is no trained weight')
        cfg['pretrain'] = False

if cfg['dropout']:
    cfg['keep_prob'] = 0.3

if cfg['A_split']:
    cfg['A_fold'] = 32

## DataSet

In [4]:
class BasicDataset(Dataset):
    def __init__(self):
        print("init dataset")
    
    @property
    def n_users(self):
        raise NotImplementedError
    
    @property
    def m_items(self):
        raise NotImplementedError
    
    @property
    def allPos(self):
        raise NotImplementedError
    
    @property
    def train_data_size(self):
        raise NotImplementedError

    @property
    def testDict(self):
        raise NotImplementedError
    
    def getUserItemFeedback(self, users, items):
        raise NotImplementedError
    
    def getUserPosItems(self, users):
        raise NotImplementedError
    
    def getUserNegItems(self, users):
        """
        not necessary for large dataset
        it's stupid to return all neg items in super large dataset
        """
        raise NotImplementedError
    
    def getSparseGraph(self):
        """
        build a graph in torch.sparse.IntTensor.
        Details in NGCF's matrix form
        A = 
            |I,   R|
            |R^T, I|
        """
        raise NotImplementedError

class BOJDataSet(BasicDataset):
    def __init__(self, config):
        print("init dataset")
        self.batch_size = config['batch_size']
        self.path = config['dataset_dir_path']
        self.adj_mat_name = config['adj_mat_name']
        self.d_mat_name = config['d_mat_name']

        train_path = self.path+'/'+ "train_" + config['dataset_file_name']
        test_path = self.path+'/'+ "test_" + config['dataset_file_name']
        self.train_user_item_mat = pd.read_csv(train_path, index_col = 0).to_numpy()
        self.test_user_item_mat = pd.read_csv(test_path, index_col = 0).to_numpy()
                        
        self.nz_train_user_item = self.train_user_item_mat.nonzero()
        self._train_data_size = self.nz_train_user_item[0].shape[0]
        self.n_user = self.train_user_item_mat.shape[0]
        self.m_item = self.train_user_item_mat.shape[1]
        
        self.split = config['A_split']
        if self.split:
            self.folds = config['A_fold']
        self.device = config['device']        
        
        print('make testDict')
        self.maxK = max(config['topks'])
        self.n_test_user = self.test_user_item_mat.shape[0]
        
        self._allPos = self.getUserPosItems(list(range(self.n_user)))
        self.__testDict = self.__build_test()
        self.__trainDict = self.__build_test(test=False)

        print(f"train data size : {self._train_data_size}")
        print(f"test user size : {self.n_test_user}")
        
        self.Graph = None
        
    @property
    def n_users(self):
        return self.n_user
    
    @property
    def m_items(self):
        return self.m_item
    
    @property
    def train_data_size(self):
        return self._train_data_size
    
    @property
    def allPos(self):
        return self._allPos

    @property
    def testDict(self):
        return self.__testDict
    
    @property
    def trainDict(self):
        return self.__trainDict
    
    # 유저가 푼 문제번호 반환
    def getUserPosItems(self, users):
        posItems = []
        for user in users:
            posItems.append(self.train_user_item_mat[user].nonzero()[0])
        return posItems
        
    def _split_A_hat(self,A):
        A_fold = []
        fold_len = (self.n_users + self.m_items) // self.folds
        for i_fold in range(self.folds):
            start = i_fold*fold_len
            if i_fold == self.folds - 1:
                end = self.n_users + self.m_items
            else:
                end = (i_fold + 1) * fold_len
            A_fold.append(self._convert_sp_mat_to_sp_tensor(A[start:end]).coalesce().to(self.device))
        return A_fold

    def _convert_sp_mat_to_sp_tensor(self, X):
        coo = X.tocoo().astype(np.float32)
        row = torch.Tensor(coo.row).long()
        col = torch.Tensor(coo.col).long()
        index = torch.stack([row, col])
        data = torch.FloatTensor(coo.data)
        return torch.sparse.FloatTensor(index, data, torch.Size(coo.shape))
        
    def getSparseGraph(self):
        print("loading adjacency matrix")
        if self.Graph is None:
            try:
                pre_adj_mat = sp.load_npz(self.path + '/'+self.adj_mat_name)
                print("successfully loaded...")
                self.d_mat = sp.load_npz(self.path + '/'+self.d_mat_name)
                norm_adj = pre_adj_mat
            except :
                print("generating adjacency matrix")
                s = time.time()
                adj_mat = sp.dok_matrix((self.n_users + self.m_items, self.n_users + self.m_items), dtype=np.float32)
                adj_mat = adj_mat.tolil()
                R = self.train_user_item_mat
                adj_mat[:self.n_users, self.n_users:] = R
                adj_mat[self.n_users:, :self.n_users] = R.T
                adj_mat = adj_mat.todok()
                # adj_mat = adj_mat + sp.eye(adj_mat.shape[0])
                
                rowsum = np.array(adj_mat.sum(axis=1))
                d_inv = np.power(rowsum, -0.5).flatten()
                d_inv[np.isinf(d_inv)] = 0.
                d_mat = sp.diags(d_inv)
                
                # 평가를 위한 mat 저장      
                self.d_mat = d_mat
                sp.save_npz(self.path + '/'+self.d_mat_name, self.d_mat)
                
                norm_adj = d_mat.dot(adj_mat) # 유저가 시도한 문제 개수 제곱근으로 나눠주기
                norm_adj = norm_adj.dot(d_mat) # 문제가 시도되어진 개수 제곱근으로 나눠주기
                norm_adj = norm_adj.tocsr()
                end = time.time()
                print(f"costing {end-s}s, saved norm_mat...")
                sp.save_npz(self.path + '/'+self.adj_mat_name, norm_adj)

            if self.split == True:
                self.Graph = self._split_A_hat(norm_adj)
                print("done split matrix")
            else:
                self.Graph = self._convert_sp_mat_to_sp_tensor(norm_adj)
                self.Graph = self.Graph.coalesce().to(self.device)
                print("don't split the matrix")
        return self.Graph

    def getEvalSparseGraph(self, users):
        graph = None
        adj_mat = sp.dok_matrix((len(users), self.n_users + self.m_items), dtype=np.float32)
        adj_mat = adj_mat.tolil()
        adj_mat[:, self.n_users:] = self.test_user_item_mat[users]
        adj_mat = adj_mat.todok()                
        
        rowsum = np.array(adj_mat.sum(axis=1))
        d_inv = np.power(rowsum, -0.5).flatten()
        d_inv[np.isinf(d_inv)] = 0.
        d_mat = sp.diags(d_inv)

        norm_adj = d_mat.dot(adj_mat) # 유저가 시도한 문제 개수 제곱근으로 나눠주기
        norm_adj = norm_adj.dot(self.d_mat) # 문제가 시도되어진 개수 제곱근으로 나눠주기
        norm_adj = norm_adj.tocsr()

        if self.split == True:
            graph = self._split_A_hat(norm_adj)
        else:
            graph = self._convert_sp_mat_to_sp_tensor(norm_adj)
            graph = graph.coalesce().to(self.device)
        return graph
    
    def __build_test(self, test=True):
        """
        return:
            dict: {user: [items]}
        """        
        
        if test:
            user_item_mat = self.test_user_item_mat
            n = self.n_test_user
        else:
            user_item_mat = self.train_user_item_mat
            n = self.n_user
        
        test_data = {}
        for user in range(n):      
            items = user_item_mat[user].nonzero()[0]
            if len(items) >= self.maxK :
                test_data[user] = items
        
        print("complete making test dict")
        
        return test_data

## Model

In [15]:
class BasicModel(nn.Module):    
    def __init__(self):
        super(BasicModel, self).__init__()
    
    def getUsersRating(self, users):
        raise NotImplementedError

        
class PairWiseModel(BasicModel):
    def __init__(self):
        super(PairWiseModel, self).__init__()
    def bpr_loss(self, users, pos, neg):
        """
        Parameters:
            users: users list 
            pos: positive items for corresponding users
            neg: negative items for corresponding users
        Return:
            (log-loss, l2-loss)
        """
        raise NotImplementedError
        
class LightGCN(PairWiseModel):
    def __init__(self, 
                 config:dict, 
                 dataset:BasicDataset):
        super(LightGCN, self).__init__()
        self.config = config
        if self.config['dropout']:
            self.keep_prob = self.config['keep_prob']
        self.dataset : dataloader.BasicDataset = dataset
        self.epoch = 0
        self.__init_weight()

    def __init_weight(self):
        self.num_users  = self.dataset.n_users
        self.num_items  = self.dataset.m_items
        self.embedding_dim = self.config['embedding_dim']
        self.n_layers = self.config['n_layers']
        self.A_split = self.config['A_split']
        self.embedding_user = torch.nn.Embedding(
            self.num_users, self.embedding_dim)
        self.embedding_item = torch.nn.Embedding(
            self.num_items, self.embedding_dim)
        if self.config['pretrain'] == 0:
            nn.init.normal_(self.embedding_user.weight, std=0.1)
            nn.init.normal_(self.embedding_item.weight, std=0.1)
            print('use NORMAL distribution initilizer')
        else:
            self.embedding_user.weight.data.copy_(self.config['user_emb'])
            self.embedding_item.weight.data.copy_(self.config['item_emb'])
            print('use pretarined data')
        self.f = nn.Sigmoid()
        self.Graph = self.dataset.getSparseGraph()
        print(f"lgn is already to go(dropout:{self.config['dropout']})")

    def __dropout_x(self, x, keep_prob):
        size = x.size()
        index = x.indices().t()
        values = x.values()
        random_index = torch.rand(len(values)) + keep_prob
        random_index = random_index.int().bool()
        index = index[random_index]
        values = values[random_index]/keep_prob
        g = torch.sparse.FloatTensor(index.t(), values, size)
        return g
    
    def __dropout(self, keep_prob):
        if self.A_split:
            graph = []
            for g in self.Graph:
                graph.append(self.__dropout_x(g, keep_prob))
        else:
            graph = self.__dropout_x(self.Graph, keep_prob)
        return graph
    
    def computer(self):
        """
        propagate methods for lightGCN
        """                    
        users_emb = self.embedding_user.weight
        items_emb = self.embedding_item.weight
        
        if self.config['dropout']:
            #print("droping")
            graph = self.__dropout(self.keep_prob)
        else:
            graph = self.Graph   
            
        all_emb = torch.cat([users_emb, items_emb])
        embs = [all_emb]
        
        
        for layer in range(self.n_layers):
            if self.A_split:
                temp_emb = []
                for f in range(len(graph)):
                    temp_emb.append(torch.sparse.mm(graph[f], all_emb))
                side_emb = torch.cat(temp_emb, dim=0)
                all_emb = side_emb
            else:
                all_emb = torch.sparse.mm(graph, all_emb)
            embs.append(all_emb)
        embs = torch.stack(embs, dim=1)
        #print(embs.size())
        light_out = torch.mean(embs, dim=1)
        users, items = torch.split(light_out, [self.num_users, self.num_items])
        return users, items
    
    def getUsersRating(self, users):
        all_users, all_items = self.computer()
        users_emb = all_users[users.long()]
        items_emb = all_items
        
        rating = self.f(torch.matmul(users_emb, items_emb.t()))
            
        return rating
    
    def getEmbedding(self, users, pos_items, neg_items):
        all_users, all_items = self.computer()
        users_emb = all_users[users]
        pos_emb = all_items[pos_items]
        neg_emb = all_items[neg_items]
        users_emb_ego = self.embedding_user(users)
        pos_emb_ego = self.embedding_item(pos_items)
        neg_emb_ego = self.embedding_item(neg_items)
        return users_emb, pos_emb, neg_emb, users_emb_ego, pos_emb_ego, neg_emb_ego
    
    def bpr_loss(self, users, pos, neg):
        (users_emb, pos_emb, neg_emb, 
        userEmb0,  posEmb0, negEmb0) = self.getEmbedding(users.long(), pos.long(), neg.long())
        reg_loss = (1/2)*(userEmb0.norm(2).pow(2) + 
                         posEmb0.norm(2).pow(2)  +
                         negEmb0.norm(2).pow(2))/float(len(users))
        pos_scores = torch.mul(users_emb, pos_emb)
        pos_scores = torch.sum(pos_scores, dim=1)
        neg_scores = torch.mul(users_emb, neg_emb)
        neg_scores = torch.sum(neg_scores, dim=1)
        
        loss = torch.mean(torch.nn.functional.softplus(neg_scores - pos_scores))
        
        return loss, reg_loss
       
    def forward(self, users, items):
        # compute embedding
        all_users, all_items = self.computer()
        # print('forward')
        #all_users, all_items = self.computer()
        users_emb = all_users[users]
        items_emb = all_items[items]
        inner_pro = torch.mul(users_emb, items_emb)
        gamma     = torch.sum(inner_pro, dim=1)
        return gamma
    
class BPRLoss:
    def __init__(self,
                 recmodel : PairWiseModel,
                 config : dict):
        self.model = recmodel
        self.weight_decay = config['decay']
        self.lr = config['lr']
        self.opt = torch.optim.Adam(recmodel.parameters(), lr=self.lr)

    def stageOne(self, users, pos, neg):
        loss, reg_loss = self.model.bpr_loss(users, pos, neg)
        reg_loss = reg_loss*self.weight_decay
        loss = loss + reg_loss

        self.opt.zero_grad()
        loss.backward()
        self.opt.step()

        return loss.cpu().item()

## Utils

In [16]:
import multiprocessing
CORES = multiprocessing.cpu_count() // 2

#########################################
################# Test ##################
#########################################

def getLabel(test_data, pred_data):
    r = []
    for i in range(len(test_data)):
        groundTrue = test_data[i]
        predictTopK = pred_data[i]
        pred = list(map(lambda x: x in groundTrue, predictTopK))
        pred = np.array(pred).astype("float")
        r.append(pred)
    return np.array(r).astype('float')

def NDCGatK_r(test_data,r,k):
    """
    Normalized Discounted Cumulative Gain
    rel_i = 1 or 0, so 2^{rel_i} - 1 = 1 or 0
    """
    assert len(r) == len(test_data)
    pred_data = r[:, :k]

    test_matrix = np.zeros((len(pred_data), k))
    for i, items in enumerate(test_data):
        length = k if k <= len(items) else len(items)
        test_matrix[i, :length] = 1
    max_r = test_matrix
    idcg = np.sum(max_r * 1./np.log2(np.arange(2, k + 2)), axis=1)
    dcg = pred_data*(1./np.log2(np.arange(2, k + 2)))
    dcg = np.sum(dcg, axis=1)
    idcg[idcg == 0.] = 1.
    ndcg = dcg/idcg
    ndcg[np.isnan(ndcg)] = 0.
    return np.sum(ndcg)

def RecallPrecision_ATk(test_data, r, k):
    """
    test_data should be a list? cause users may have different amount of pos items. shape (test_batch, k)
    pred_data : shape (test_batch, k) NOTE: pred_data should be pre-sorted
    k : top-k
    """
    right_pred = r[:, :k].sum(1)
    precis_n = k
    recall_n = np.array([len(test_data[i]) for i in range(len(test_data))])
    recall = np.sum(right_pred/recall_n)
    precis = np.sum(right_pred)/precis_n
    return {'recall': recall, 'precision': precis}


def test_one_batch(X):
    sorted_items = X[0].numpy()
    groundTrue = X[1]
    r = getLabel(groundTrue, sorted_items)
    pre, recall, ndcg = [], [], []
    for k in cfg['topks']:
        ret = RecallPrecision_ATk(groundTrue, r, k)
        pre.append(ret['precision'])
        recall.append(ret['recall'])
        ndcg.append(NDCGatK_r(groundTrue,r,k))
    return {'recall':np.array(recall), 
            'precision':np.array(pre), 
            'ndcg':np.array(ndcg)}
            
def Test(dataset : BasicDataset, Recmodel : LightGCN, cfg):
    u_batch_size = cfg['test_batch_size']
    print('[----- Train Data -----]')
    testDict: dict = dataset.trainDict
    # eval mode with no dropout
    Recmodel = Recmodel.eval()
    max_K = max(cfg['topks'])
    results = {'precision': np.zeros(len(cfg['topks'])),
               'recall': np.zeros(len(cfg['topks'])),
               'ndcg': np.zeros(len(cfg['topks']))}
    with torch.no_grad():
        users = list(testDict.keys())
        try:
            assert u_batch_size <= len(users) / 10
        except AssertionError:
            print(f"test_u_batch_size is too big for this dataset, try a small one {len(users) // 10}")
        users_list = []
        rating_list = []
        groundTrue_list = []
        # auc_record = []
        # ratings = []
        total_batch = len(users) // u_batch_size + 1
        for batch_users in tqdm(minibatch(users, batch_size=u_batch_size)):
            groundTrue = [testDict[u] for u in batch_users]
            _batch_users = torch.Tensor(batch_users).long()
            rating = Recmodel.getUsersRating(_batch_users)
            #rating = rating.cpu()
            print('Exclude Rated Item')
            allPos = dataset.getUserPosItems(batch_users)
            exclude_index = []
            exclude_items = []
            for range_i, items in enumerate(allPos):
                exclude_index.extend([range_i] * len(items))
                exclude_items.extend(items)
            rating[exclude_index, exclude_items] = -(1<<10)
            _, rating_K = torch.topk(rating, k=max_K)
            rating = rating.cpu().numpy()
            # aucs = [ 
            #         utils.AUC(rating[i],
            #                   dataset, 
            #                   test_data) for i, test_data in enumerate(groundTrue)
            #     ]
            # auc_record.extend(aucs)
            del rating
            users_list.append(batch_users)
            rating_list.append(rating_K.cpu())
            groundTrue_list.append(groundTrue)
        assert total_batch == len(users_list)
        X = zip(rating_list, groundTrue_list)
        pre_results = []
        for x in X:
            pre_results.append(test_one_batch(x))
        scale = float(u_batch_size/len(users))
        for result in pre_results:
            results['recall'] += result['recall']
            results['precision'] += result['precision']
            results['ndcg'] += result['ndcg']
        results['recall'] /= float(len(users))
        results['precision'] /= float(len(users))
        results['ndcg'] /= float(len(users))
        # results['auc'] = np.mean(auc_record)
        print(results)
        return results

#########################################
################# TRAIN #################
#########################################

def minibatch(*tensors, **kwargs):

    batch_size = kwargs.get('batch_size', cfg['batch_size'])

    if len(tensors) == 1:
        tensor = tensors[0]
        for i in range(0, len(tensor), batch_size):
            yield tensor[i:i + batch_size]
    else:
        for i in range(0, len(tensors[0]), batch_size):
            yield tuple(x[i:i + batch_size] for x in tensors)

def shuffle(*arrays, **kwargs):

    require_indices = kwargs.get('indices', False)

    if len(set(len(x) for x in arrays)) != 1:
        raise ValueError('All inputs to shuffle must have '
                         'the same length.')

    shuffle_indices = np.arange(len(arrays[0]))
    np.random.shuffle(shuffle_indices)

    if len(arrays) == 1:
        result = arrays[0][shuffle_indices]
    else:
        result = tuple(x[shuffle_indices] for x in arrays)

    if require_indices:
        return result, shuffle_indices
    else:
        return result 

def UniformSample_original_python(dataset : BasicDataset):
    """
    the original impliment of BPR Sampling in LightGCN
    :return:
        np.array
    """
    user_num = dataset.train_data_size
    
    users = np.random.randint(0, dataset.n_users, user_num)
    allPos = dataset.allPos
    S = []
    for i, user in enumerate(users):
        posForUser = allPos[user]
        if len(posForUser) == 0:
            continue
        posindex = np.random.randint(0, len(posForUser))
        positem = posForUser[posindex]
        while True:
            negitem = np.random.randint(0, dataset.m_items)
            if negitem in posForUser:
                continue
            else:
                break
        S.append([user, positem, negitem])
    return np.array(S)
    
def BPR_train_original(dataset, recommend_model, loss_class : BPRLoss, cfg, neg_k=1):
    Recmodel = recommend_model
    Recmodel.train()
    bpr = loss_class
    
    S = UniformSample_original_python(dataset)
    users = torch.Tensor(S[:, 0]).long()
    posItems = torch.Tensor(S[:, 1]).long()
    negItems = torch.Tensor(S[:, 2]).long()

    users = users.to(cfg['device'])
    posItems = posItems.to(cfg['device'])
    negItems = negItems.to(cfg['device'])
    users, posItems, negItems = shuffle(users, posItems, negItems)
    total_batch = len(users) // cfg['batch_size'] + 1
    aver_loss = 0.
    print(total_batch)
    for (batch_i, (batch_users, batch_pos, batch_neg)) in tqdm(enumerate(minibatch(users, posItems, negItems,
                                                                    batch_size=cfg['batch_size']))):
        cri = bpr.stageOne(batch_users, batch_pos, batch_neg)
        aver_loss += cri
    aver_loss = aver_loss / total_batch
    return f"loss{aver_loss:.3f}"

## Train & Test

In [17]:
# declare dataset, model, loss func
dataset = BOJDataSet(cfg)
Light_GCN = LightGCN(cfg, dataset).cuda()
bpr_loss = BPRLoss(Light_GCN, cfg)

init dataset
make testDict
complete making test dict
complete making test dict
train data size : 497076
test user size : 750
use NORMAL distribution initilizer
loading adjacency matrix
successfully loaded...
don't split the matrix
lgn is already to go(dropout:False)


In [18]:
# Train & Test
st_epoch = 0
TRAIN_epochs = 100
start = time.time()
for epoch in range(st_epoch, st_epoch + TRAIN_epochs):
    if epoch % 10 == 0:
        print('[Test]')
        Test(dataset, Light_GCN, cfg) # test data로 평가
        Test(dataset, Light_GCN, cfg, test=False) # train data로 평가
        torch.save(Light_GCN.state_dict(), cfg['model_file_path'])
    
    output_information = BPR_train_original(dataset, Light_GCN, bpr_loss, cfg)
    print(f'EPOCH[{epoch+1}/{st_epoch + TRAIN_epochs}] {output_information}')
torch.save(Light_GCN.state_dict(), cfg['model_file_path']+f"_{st_epoch + TRAIN_epochs}")
total_time = time.time() - start
print(f'total_time : {total_time}, loss : {output_information}')

[Test]
[----- Test Data -----]


0it [00:01, ?it/s]


RuntimeError: addmm: Argument #3 (dense): Expected dim 0 size 28732, got 32

In [114]:
# Test
Test(dataset, Light_GCN, cfg)
Test(dataset, Light_GCN, cfg, test=False)

[----- Test Data -----]


24it [00:01, 14.13it/s]


{'precision': array([0.85546667, 0.84113333]), 'recall': array([0.03442167, 0.06760341]), 'ndcg': array([0.8691201 , 0.85409283])}
[----- Train Data -----]


55it [00:00, 69.35it/s]


{'precision': array([0.87508571, 0.85405714]), 'recall': array([0.03412683, 0.06642766]), 'ndcg': array([0.8867245, 0.8678895])}


{'precision': array([0.87508571, 0.85405714]),
 'recall': array([0.03412683, 0.06642766]),
 'ndcg': array([0.8867245, 0.8678895])}

In [58]:
Test(dataset, Light_GCN, cfg)

24it [00:01, 12.37it/s]

{'precision': array([0.87666667, 0.85413333]), 'recall': array([0.0353388 , 0.06870723]), 'ndcg': array([0.88961997, 0.86918327])}


{'precision': array([0.87666667, 0.85413333]),
 'recall': array([0.0353388 , 0.06870723]),
 'ndcg': array([0.88961997, 0.86918327])}

In [19]:
import os
lgcn_model_dir = './saved_model'
lgcn_model_file_name = 'lgcn'
model_link_path = os.path.join(lgcn_model_dir,lgcn_model_file_name)

os.chmod(cfg['model_file_path'], 0o777)

with open(cfg['model_file_path'], 'wb') as file:  # 파일을 바이너리 쓰기 모드(wb)로 열기
    if os.path.islink(model_link_path):
        os.unlink(model_link_path)
    os.symlink(cfg['model_file_path'], model_link_path)

OSError: [WinError 1314] 클라이언트가 필요한 권한을 가지고 있지 않습니다: './saved_model/lgcn_weight_10.pt' -> './saved_model\\lgcn'

In [9]:
Light_GCN.state_dict()

NameError: name 'Light_GCN' is not defined